# Overview of the Settings Attribute

OpenPNM objects all include a ``settings`` attribute which contains certain information used by OpenPNM. The best example is the ``algorithm`` classes, which often require numerous settings such as number of iterations and tolerance for iterative calculations.  This tutorial will provide an overview of how these settings work, both from the user perspective as well as for developers.

In [1]:
import openpnm as op
pn = op.network.Cubic([4, 4,])
geo = op.geometry.SpheresAndCylinders(network=pn, pores=pn.Ps, throats=pn.Ts)
air = op.phases.Air(network=pn)
phys = op.physics.Basic(network=pn, phase=air, geometry=geo)

## Normal Usage

> This section is relevant to users of OpenPNM, while the next section is more relevant to developers

Let's look an algorithm that has numerous settings:

In [2]:
alg = op.algorithms.ReactiveTransport(network=pn, phase=air)

We can see that many default settings are already present by printing the ``settings`` attribute:

In [3]:
print(alg.settings)

――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――
Settings                            Values
――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――
name                                react_trans_01
prefix                              react_trans
uuid                                443dbba7-987e-4649-bf75-fa1ae988a54f
bob                                 5
f_rtol                              1e-06
newton_maxiter                      5000
nlin_max_iter                       5000
relaxation_quantity                 1.0
relaxation_source                   1.0
sources                             []
x_rtol                              1e-06
cache_A                             True
cache_b                             True
conductance                         
phase                               phase_01
quantity                            
――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――


We can override these settings manually:

In [4]:
alg.settings.prefix = 'rxn'
alg.settings.prefix

'rxn'

We could also have updated these settings when creating the algorithm object by passing in a set of arguments.  This can be in the form of a dictionary:

In [5]:
s = {"prefix": "rxn"}
alg = op.algorithms.ReactiveTransport(network=pn, phase=air, settings=s)
print(alg.settings)

――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――
Settings                            Values
――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――
name                                rxn_02
prefix                              rxn
uuid                                98d0913d-0ffd-4d5e-8c9d-6b5354a7aa57
bob                                 5
f_rtol                              1e-06
newton_maxiter                      5000
nlin_max_iter                       5000
relaxation_quantity                 1.0
relaxation_source                   1.0
sources                             []
x_rtol                              1e-06
cache_A                             True
cache_b                             True
conductance                         
phase                               phase_01
quantity                            
――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――


Or as a 'dataclass' style, which is how things are done behind the scenes in OpenPNM as described in the section:

In [6]:
class MySettings:
    prefix = 'rxn'
    new_setting = 44.44
    
    
alg = op.algorithms.ReactiveTransport(network=pn, phase=air, settings=MySettings())
print(alg.settings)

――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――
Settings                            Values
――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――
name                                rxn_03
prefix                              rxn
uuid                                8735d6dc-8162-449f-8943-ed1214565e16
bob                                 5
f_rtol                              1e-06
newton_maxiter                      5000
nlin_max_iter                       5000
relaxation_quantity                 1.0
relaxation_source                   1.0
sources                             []
x_rtol                              1e-06
new_setting                         44.44
cache_A                             True
cache_b                             True
conductance                         
phase                               phase_01
quantity                            
―――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――

One new feature on OpenPNM V3 is that the datatype of the settings is enforced.  For instance the ``'prefix'`` setting must be a ``str``, otherwise an error is raised:

In [7]:
try:
    alg.settings.phase = 1
except TypeError as e:
    print(e)

Attribute 'phase' can only accept values of type <class 'str'>, but the recieved value was of type <class 'int'>


This is very useful for the backend of OpenPNM to ensure that the ``settings`` are always in the correct and expected format.  

## Advanced Usage

> The following sections are probably only relevant if you plan to do some development in OpenPNM

In the previous section we saw how to define settings, as well as the data-type protections of settings.  In this section we'll demonstrate this mechanism in more detail.

The ``settings`` attribute of all OpenPNM objects is an instance of the new ``SettingsAttr`` class.  It is inspired by the new [Python dataclass](https://docs.python.org/3/library/dataclasses.html) but offers more an additional key feature: ``dataclasses`` allow developers to specify the type of attributes (i.e. ``obj.a`` must be an ``int``), but these are only enforced during object creation. Once the object is made, any value can be assigned to ``a``.  The ``SettingsAttr`` class enforces the type of ``a`` for all subsequent assignments.  We saw this in action in the previous section when we tried to assign an integer to ``alg.sets.prefix``. Another feature of the ``SettingsAttr`` is that the types do not need to be specified, but are instead inferred from the value assigned to them.  This will be demonstrated below.

### Defining Custom Settings and Adding to a SettingsAttr Object

Let's first create an empty ``SettingsAttr`` instance, then ``_update`` it with some custom values:

In [8]:
from openpnm.utils import SettingsAttr

In [9]:
class CustomSettings:
    a = 0
    b = 4.4
    c = {}
    d = []
    
s = SettingsAttr()
s._update(CustomSettings)

Now we can print ``s`` to inspect the settings.  We'll see some default values for things that were not initialized like ``a``, while ``b`` is the specified value.

In [10]:
print(s)

――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――
Settings                            Values
――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――
a                                   0
b                                   4.4
c                                   {}
d                                   []
――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――


We used the hidden ``_update`` method to add the custom settings the ``SettingsAttr`` instance.  This method is hidden because we wanted to keep the namespace completely dedicated to the actual settings, instead of mixing settings and methods. The ``_update`` method is used within the init of all the OpenPNM objects to first update the ``settings`` attribute with the default settings for the class, and then to apply the user supplied settings.  This is demonstrated below:

In [28]:
from openpnm.core import Base

class DefaultSettings:
    name = 'foo'
    a = 1
    b = 'string'
    

class some_algorithm(Base):
    def __init__(self, settings={}, **kwargs):
        # Add default settings defined internally
        self.settings._update(DefaultSettings)
        # Upddate with any user-supplied settings
        self.settings._update(settings)
        
b = some_algorithm(settings={'b': 'hello', 'c': 4.4})
print(b.settings)

――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――
Settings                            Values
――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――
name                                foo
prefix                              base
uuid                                614ed51f-bd28-4137-aec2-0868271f294f
a                                   1
b                                   hello
c                                   4.4
――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――


### Enforced Data-Types

The ``SettingsAttr`` class enforces the datatype of each of these attributes:

In [12]:
s.a = 2
s.b = 5.5
print(s)

――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――
Settings                            Values
――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――
a                                   2
b                                   5.5
c                                   {}
d                                   []
――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――


Let's look at the attribute protection in action again:

In [13]:
try:
    s.a = 1.1
except TypeError as e:
    print(e)

Attribute 'a' can only accept values of type <class 'int'>, but the recieved value was of type <class 'float'>


Also, we can't accidentally overwrite an attribute that is supposed to be a list with a scalar:

In [14]:
try:
    s.d = 5
except TypeError as e:
    print(e)

Attribute 'd' can only accept values of type <class 'list'>, but the recieved value was of type <class 'int'>


OpenPNM also provides a class called ``TypedList`` which enforces what each element of a list can be.  This is to ensure that a list only contains strings, for instance:

In [15]:
from openpnm.utils import TypedList

class MySettings:
    fruits = TypedList(['apple', 'banana'])
    
s = SettingsAttr()
s._update(MySettings)
print(s)

――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――
Settings                            Values
――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――
fruits                              ['apple', 'banana']
――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――


In [16]:
try:
    s.fruits[0] = 55
except TypeError as e:
    print(e)

This list cannot accept values of type <class 'int'>


## Adding Documentation to a SettingsAttr Class

One the main reasons for using a ``dataclass`` style object for holding settings is so that docstrings for each attribute can be defined and explained:

In [19]:
class DocumentedSettingsData:
    r"""
    A class that holds the following settings.
    
    Parameters
    ----------
    name : str
        The name of the object
    id_num : int
        The id number of the object
    """
    name = 'foo'
    id_num = 0

d = DocumentedSettingsData()

In [20]:
print(d.__doc__)


    A class that holds the following settings.
    
    Parameters
    ----------
    name : str
        The name of the object
    id_num : int
        The id number of the object
    


Note that this docstring was written when we defined ``DocumentedSettingsData`` subclass and it attached to it, but we'll be interacting with the ``SettingsAttr`` class.  When a ``SettingsAttr`` is created is adopts the docstring of the received ``settings`` object.  This can be either a proper ``SettingsData/HasTraits`` class or a basic ``dataclass`` style object.  The docstring can only be set on initialization though, so any new attributes that are created by adding values to the object (i.e. ``D.zz_top = 'awesome'``) will not be documented.

In [21]:
D = SettingsAttr(d)
print(D.__doc__)


    A class that holds the following settings.
    
    Parameters
    ----------
    name : str
        The name of the object
    id_num : int
        The id number of the object
    


This machinery was designed with the idea of inheriting docstrings using the ``docrep`` package.  The following illustrates not only how the ``SettingsData`` class can be subclassed to add new settings (e.g. from ``GenericTransport`` to ``ReactiveTransport``), but also how to use the hightly under-rated ``docrep`` package to also inherit the docstrings:

In [24]:
import docrep
docstr = docrep.DocstringProcessor()


# This docorator tells docrep to fetch the docstring from this class and make it available elsewhere:
@docstr.get_sections(base='DocumentSettingsData', sections=['Parameters'])
class DocumentedSettingsData:
    r"""
    A class that holds the following settings.
    
    Parameters
    ----------
    name : str
        The name of the object
    id_num : int
        The id number of the object
    """
    name = 'foo'
    id_num = 0


# This tells docrep to parse this docstring and insert text at the %
@docstr.dedent
class ChildSettingsData:
    r"""
    A subclass of DocumentedSettingsData that holds some addtional settings
    
    Parameters
    ----------
    %(DocumentSettingsData.parameters)s
    max_iter : int
        The maximum number of iterations to do
    """
    max_iter = 10

    
E = ChildSettingsData()
print(E.__doc__)

A subclass of DocumentedSettingsData that holds some addtional settings

Parameters
----------
name : str
    The name of the object
id_num : int
    The id number of the object
max_iter : int
    The maximum number of iterations to do


Again, as mentioned above, this inherited docstring is adopted by the ``SettingsAttr``:

In [27]:
S = SettingsAttr(E)
print(S.__doc__)

A subclass of DocumentedSettingsData that holds some addtional settings

Parameters
----------
name : str
    The name of the object
id_num : int
    The id number of the object
max_iter : int
    The maximum number of iterations to do
